In [7]:
import openrouteservice as ors
import folium
import json

# Charger la clé API depuis un fichier
API = open("OpenRouteService.txt", "r").read()
client = ors.Client(key=API)

# Coordonnées des utilisateurs
coords_user1 = [[6.183264, 48.693684], [6.172724, 48.667496]]
coords_user2 = [[6.183264, 48.693684], [6.172724, 48.667496]]
coords_user3 = [[6.175689, 48.697079], [6.142902, 48.679287]]
coords_user4 = [[6.171398, 48.712372], [6.204700, 48.710333]]
coords_user5 = [[6.156463, 48.686314], [6.178264, 48.672939]]
coords_user6 = [[6.165390, 48.686767], [6.169338, 48.666137]]

# Obtenir les itinéraires pour chaque utilisateur
route_user1 = client.directions(coordinates=coords_user1, profile='cycling-regular', format='geojson')
route_user2 = client.directions(coordinates=coords_user2, profile='cycling-regular', format='geojson')
route_user3 = client.directions(coordinates=coords_user3, profile='cycling-regular', format='geojson')
route_user4 = client.directions(coordinates=coords_user4, profile='cycling-regular', format='geojson')
route_user5 = client.directions(coordinates=coords_user5, profile='cycling-regular', format='geojson')
route_user6 = client.directions(coordinates=coords_user6, profile='cycling-regular', format='geojson')

# Fonction pour trouver la plus longue sous-séquence commune entre deux listes
def longest_common_subsequence(list1, list2):
    m = len(list1)
    n = len(list2)
    
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if list1[i - 1] == list2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    lcs_length = dp[m][n]
    lcs = []
    i, j = m, n
    
    while i > 0 and j > 0:
        if list1[i - 1] == list2[j - 1]:
            lcs.append(list1[i - 1])
            i -= 1
            j -= 1
        elif dp[i - 1][j] > dp[i][j - 1]:
            i -= 1
        else:
            j -= 1
    
    return lcs[::-1]

# Fonction pour comparer deux tracés de chemin et déterminer la similarité
def compare_routes(route1, route2):
    coords1 = route1['features'][0]['geometry']['coordinates']
    coords2 = route2['features'][0]['geometry']['coordinates']
    
    lcs = longest_common_subsequence(coords1, coords2)
    similarity = len(lcs) / min(len(coords1), len(coords2))
    
    return lcs, similarity

# Créer un dictionnaire pour stocker les paires d'utilisateurs similaires
similar_users = {}

# Comparer tous les utilisateurs entre eux
for i in range(len([route_user1, route_user2, route_user3, route_user4,route_user5,route_user6])):
    for j in range(i + 1, len([route_user1, route_user2, route_user3, route_user4,route_user5,route_user6])):
        user1 = [route_user1, route_user2, route_user3, route_user4,route_user5,route_user6][i]
        user2 = [route_user1, route_user2, route_user3, route_user4,route_user5,route_user6][j]
        
        lcs, similarity = compare_routes(user1, user2)
        if similarity > 0:  # Seuil de similarité 
            key = f"User{i + 1} et User{j + 1}"
            similar_users[key] = {"similarity": similarity, "lcs": lcs}

# Afficher les paires d'utilisateurs similaires
print("Utilisateurs avec des parties de chemin communes :")
for pair, data in similar_users.items():
    similarity = data["similarity"]
    lcs = data["lcs"]
    
    start_coords = lcs[0]
    end_coords = lcs[-1]
    
    print(f"{pair} - Similarité : {similarity}, Début : {start_coords}, Fin : {end_coords}")

# Créer la carte avec les tracés de chemin
m = folium.Map(location=list(reversed([6.183264, 48.693684])), zoom_start=13)

# Ajouter les tracés de chemin à la carte
for i, route in enumerate([route_user1, route_user2, route_user3, route_user4,route_user5,route_user6]):
    color = ["blue", "red", "green", "purple","orange","black"][i]
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']],
                    color=color).add_to(m)
    
# Ajouter des marqueurs au début et à la fin de chaque chemin
    start_marker = folium.Marker(location=list(reversed(route['features'][0]['geometry']['coordinates'][0])),
                                popup=f"Début - User {i + 1}",
                                icon=folium.Icon(color=color))
    end_marker = folium.Marker(location=list(reversed(route['features'][0]['geometry']['coordinates'][-1])),
                              popup=f"Fin - User {i + 1}",
                              icon=folium.Icon(color=color))

    m.add_child(start_marker)
    m.add_child(end_marker)

# Afficher la carte
m


ModuleNotFoundError: No module named 'folium'